## This script is used to compare the results from all 3 software.
## Written by Yusreen Shah
## Date: May 10th 2023

In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict


## This section saves the list of all the queries in a dataframe

In [2]:
# Save the data
data = defaultdict(list)

# Read the values from the .fasta file, and save them to data
for seq_record in SeqIO.parse("combined_1.fasta", "fasta"):
    query=seq_record.id
    sequence= repr(seq_record.seq)
    length=len(seq_record)
    data['Query'].append(query)
    data['Sequence'].append(sequence)
    data['Length'].append(length)
    
# Add the data to a dataframe
df = pd.DataFrame.from_dict(data)


In [3]:
# Remove Seq(' and ') from the sequences
df['Sequence'] = df['Sequence'].str.replace('Seq(''', '')
df['Sequence'] = df['Sequence'].str.replace(')', '')
df['Sequence'] = df['Sequence'].str.strip(" \' ")
df['Sequence']=df['Sequence'].str.rstrip()

In [4]:
df=df.sort_values("Query")
df=df.reset_index(drop=True)
df

,Query,Sequence,Length
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858


## This section is used to compare the results from Bandage and the actual list of queries.

In [5]:
#Create a dataframe for Bandage Combined1
Bandage_Combined1= pd.read_csv('Bandageoutputcombined1.tsv', sep='\t')

In [6]:
Bandage_Combined1.head()

,Query,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,(56) 7593+ (642),587,95.915%,95.915%,99.83%,1,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...
1,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,(93) 7593+ (642),550,96.4912%,96.4912%,99.455%,3,0,100%,0,0,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...
2,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,(88) 7593+ (642),555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
3,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,(88) 7593+ (642),555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
4,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,(56) 7593+ (642),587,95.915%,95.915%,99.659%,2,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...


In [7]:
Bandage_Combined_Query_Path =Bandage_Combined1[['Path','Query','Sequence']]

## Extract the start and end position from the path in Bandage

In [8]:
# Extract the start position
pattern_path_start = r'\((.*?)\)' 

In [9]:
Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)
Bandage_Combined_Query_Path

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2920/3300280694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)


,Path,Query,Sequence,Start
0,(56) 7593+ (642),gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56
1,(93) 7593+ (642),gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...,93
2,(88) 7593+ (642),gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88
3,(88) 7593+ (642),gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88
4,(56) 7593+ (642),gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56
...,...,...,...,...
388,(37) 5967+ (897),gb|AJ318094|+|0-861|ARO:3000961|TEM-94,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37
389,(37) 5967+ (897),gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37
390,(37) 5967+ (897),gb|AY092401|+|0-861|ARO:3000963|TEM-96,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37
391,(33708) 1107- (35627),gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...,33708


In [10]:
#Extract the end position
Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()').astype(int)
Bandage_Combined_Query_Path

/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2920/3965281820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]
/var/folders/b6/1fdgzx1s4v13606y1h1qy1v80000gn/T/ipykernel_2920/3965281820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()').astype(int)


,Path,Query,Sequence,Start,End
0,(56) 7593+ (642),gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56,642
1,(93) 7593+ (642),gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...,93,642
2,(88) 7593+ (642),gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88,642
3,(88) 7593+ (642),gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88,642
4,(56) 7593+ (642),gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56,642
...,...,...,...,...,...
388,(37) 5967+ (897),gb|AJ318094|+|0-861|ARO:3000961|TEM-94,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897
389,(37) 5967+ (897),gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897
390,(37) 5967+ (897),gb|AY092401|+|0-861|ARO:3000963|TEM-96,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897
391,(33708) 1107- (35627),gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...,33708,35627


In [29]:
#Reorder the columns in Bandage_Combined_Query_Path
Bandage_Combined_Query_Path=Bandage_Combined_Query_Path[['Query','Path','Start','End','Sequence']]

#Bandage_Combined1['Query'].value_counts() 
# Merge the rows that have the same query
# group the dataframe by the 'Name' column and aggregate the data for each group
merged_df = Bandage_Combined_Query_Path.groupby('Query').agg({'Path': 'first', 'Sequence': ', '.join}).reset_index()
merged_df 

,Query,Path,Sequence
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,(24) 5151- (884),ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,(37) 5967+ (897),ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,(24) 5151- (884),ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,(24) 5151- (884),ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,(24) 5151- (884),ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,(37) 5967+ (897),ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,(37) 5967+ (894),ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,(37) 5967+ (894),ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,(37) 5967+ (894),ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [21]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsBandage=pd.merge(df,Bandage_Combined_Query_Path, on='Query',how="outer")

In [22]:
df_QueryAndResultsBandage

,Query,Sequence_x,Length,Path,Start,End,Sequence_y
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [23]:
for col in df_QueryAndResultsBandage.columns:
    print(col)

Query
Sequence_x
Length
Path
Start
End
Sequence_y


In [24]:
df_QueryAndResultsBandage.columns

Index(['Query', 'Sequence_x', 'Length', 'Path', 'Start', 'End', 'Sequence_y'], dtype='object')

In [25]:
df_QueryAndResultsBandage.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)

In [26]:
frames = [df, df_QueryAndResultsBandage]

In [27]:
result = pd.concat(frames)
display(result)

,Query,Sequence,Length,Path,Start,End,Sequence_Bandage
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,NaN,NaN,NaN,NaN
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897.0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894.0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894.0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894.0,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [ ]:
for col in df_QueryAndResultsBandage.columns:
    print(col)

## This section is used to compare the results from SPAligner and the actual list of queries.

In [ ]:
#Create a dataframe for SPAligner Combined1
SPAligner_Combined1=pd.read_csv('SPAligneroutputcombined1.tsv', sep='\t')

In [ ]:
#format the Query column from  SPAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


SPAligner_Combined1['Query'] = SPAligner_Combined1['Query'].map(format_query)


In [ ]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Sequence']]

In [ ]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsSPAligner=pd.merge(df,SPAligner_Combined_Query_Path, on='Query')

In [ ]:
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_y':'Sequence_SPAligner'}, inplace = True)

In [ ]:
#Create a dataframe for GraphAligner Combined1
GraphAligner_Combined1=pd.read_csv('GraphAligneroutputcombined1.tsv', sep='\t', names=["Query", "Query Length", "Query Start", 
                                          "Query End","Strand Relative Length","Path Matching","Path Length",
                                         "Start Position on Path","End Position on Path","Number of residues Matches",
                                         "Alignment Back Length","Mapping Quality","Column 1"])

In [ ]:
#Format the Query column from  GraphAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


GraphAligner_Combined1['Query'] = GraphAligner_Combined1['Query'].map(format_query)

In [ ]:
# Remove < and > from the Path
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('>', "")
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('<', "")

## This section saves the Nodes and Sequences from the gfa file

In [ ]:
import gfapy

# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [ ]:
data_graph = []

In [ ]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

In [ ]:
#Count the number of duplicates
#len(df_graph['Name'])-len(df_graph['Name'].drop_duplicates())

In [ ]:
num_segments

In [ ]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [ ]:
df_graph

In [ ]:
df_graph.loc[df_graph['Name'] == '7593']